### Background info

Followed the 3-year reliability paper: https://doi.org/10.1016/j.neuroimage.2021.118516
1. "Using these source estimates, we then estimated the power of cortical activity in the canonical frequency bands: delta (2–4 Hz), theta (4–8 Hz), alpha (8–12 Hz), beta (15–30 Hz), low gamma (30–80 Hz), and high gamma (80–150 Hz). We used Welch's method for estimating power spectrum densities (PSD) per four-second epoch across each MEG recording, with 1-second sliding Hamming windows overlapping at 50%. We then standardized the PSD values at each frequency bin to the total power across the frequency spectrum. We then averaged PSD maps (ie. source estimates) across epochs for each participant to obtain one set of six PSD maps (one per frequency band) per participant per visit."
2. "We projected these maps onto the MNI ICBM152 brain template (Fonov et al., 2009) and applied a 3 mm FWHM smoothing kernel."
3. "Used single rater two-way mixed-effects model and absolute agreement definition, or ICC(A,1)... ICC estimates and their 95% confidence intervals were calculated using the Matlab Central file-exchange ICC.m function. This ICC calculation was applied at every vertex in the PSD maps to obtain spatially specific reliability estimates at each of the frequency bands. This resulted in an ICC map per frequency band."
4. "To further visualize the reliability of source power in each frequency band, regions of interest (Brainstorm “scouts”) in the frontal, parietal, temporal, and occipital lobes were applied to each participant's PSD map. The average power (relative to total spectral power) across each lobe was extracted for each participant and each visit. ICCs of these values were then calculated using the same ICC(A,1) model."
- "ICC .5 indicates poor reliability, values between .5 to .75 indicates moderate reliability, values between .75 and .9 indicates good reliability, and values greater than .9 indicate excellent reliability. Importantly, we evaluated the level of reliability based on the 95% confidence interval of the ICC estimate, not the estimate itself, since the interval reveals the chance that the true ICC value lands on any point between the bounds."

### Bands

generic_taskfree_MEGIN.py Add this change: 
- stc_band_base = Path(os.path.basename(path2raw)).stem + f"_{inv_method}_{band}_stc"
- stc_band_path = os.path.join(stc_dir, stc_band_base)
- stc_band.save(stc_band_path, overwrite=True)

### Morph

Here (ie, in this script), create morph and apply
- The 3yr reliability paper applied a 3mm smoothing kernel, but can't do this in mne python! Caution that the smoothing param that is part of the morph function does not do what you think it does
- Save morphed_stc_per_band -> end up with 5 stcs per run per participant

Sources: 
- https://mne.tools/stable/auto_examples/inverse/morph_surface_stc.html

### ICC

**Paper trail of decisions:**

What past people have used: 
- 3-year reliability paper: single rater, two-way mixed-effects, absolute agreement (or ICC(A,1)) using matlab ICC.m
- spectral stability paper: single-rater, two-way mixed-effects, absolute agreement, using matlab ICC.m

What I will use: single rater, two-way mixed-effects, absolute agreement (or ICC(A,1))
- Based on examples above, as well as other sources: https://rowannicholls.github.io/python/statistics/agreement/intraclass_correlation.html, https://pmc.ncbi.nlm.nih.gov/articles/PMC4913118/  
- Model: Two-way mixed effects. This means same set of raters for all subjects. Mixed  means that raters are the only raters of interest; cannot be generalized to population. Note that results from two-way random effects would be the same.
- Definition: absolute agreement (how much different raters' ratings agree) (rather than consistency, which is a measure of if raters scores for same subject are correlated)

Looking into matlab ICC.m: 
- https://www.mathworks.com/matlabcentral/fileexchange/22099-intraclass-correlation-coefficient-icc 
- Based on this paper: McGraw, K. O., Wong, S. P., "Forming Inferences About Some Intraclass Correlation Coefficients", Psychological Methods, Vol. 1, No. 1, pp. 30-46, 1996  
- Looked at formula for ICC(A, 1)
- I want to use Python. Which code in the python Pingouin ICC (https://github.com/raphaelvallat/pingouin/blob/dcfdc82bbc7f1ba5991b80717a5ca156617443e8/pingouin/reliability.py#L158) corresponds to this formula? Answer: ICC2 "Single random raters"


### Set up

In [ ]:
# IMPORT PACKAGES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import pickle
import mne
import copy as copy
from pathlib import Path
import csv
import pingouin as pg
import tempfile

base_dir = "C:/meg/NVAR_ICC/derivatives"

# Things to loop through
dictionary = { 
    "NVAR008": ["251016", "251017", "251023", "251113"], 
    "NVAR010": ["251027", "251028", "251103", "251124"], 
    "NVAR011": ["251030", "251031", "251106", "251127"]
}
runs = ["rest1", "rest2"]
bands = ["delta", "theta", "alpha", "beta", "g_low", "g_high"]


### Morph

In [ ]:
for subject in dictionary: 
    for session in dictionary[subject]:
        for band in bands: 
            for run in runs: 

                # Make file name
                stc_dir = os.path.join(base_dir, "sub_" + subject, session, "beamformer", "stc")
                stc_basename = os.path.join(stc_dir, "sub_" + subject + "_" + run + "_raw_tsss_beamformer_" + band + "_stc")
                print("Input: " + stc_basename)
                print("Output: " + stc_basename + "_morphed")

                # Read in file
                stc = mne.read_source_estimate(stc_basename)

                # Create morph
                morph = mne.compute_source_morph(
                    stc,
                    subject_from = 'sub_' + subject, 
                    subject_to = "fsaverage", # to fsaverage
                    subjects_dir = "C:/meg/NVAR_ICC/MRI/freesurfer/"
                    )
                # Apply morph
                stc_morphed = morph.apply(stc)
                stc_morphed.save(stc_basename + "_morphed", overwrite = True)

In [18]:
# CHECKING

rest1 = mne.read_source_estimate("C:/meg/NVAR_ICC/derivatives/sub_NVAR008/Day1/beamformer/stc/sub_NVAR008_rest1_raw_tsss_beamformer_beta_stc_morphed")
#rest2 = mne.read_source_estimate("C:/meg/NVAR_ICC/derivatives/sub_NVAR008/251113/beamformer/stc/sub_NVAR008_rest2_raw_tsss_beamformer_beta_stc_morphed")
#average = mne.read_source_estimate("C:/meg/NVAR_ICC/derivatives/sub_NVAR008/251113/beamformer/stc/sub_NVAR008_average_raw_tsss_beamformer_beta_stc_morphed")
#og = mne.read_source_estimate("C:/meg/NVAR_ICC/derivatives/sub_NVAR008/251113/beamformer/stc/sub_NVAR008_rest1_raw_tsss_beamformer_beta_stc_morphed")

brain = rest1.plot(
    subject = "fsaverage", 
    subjects_dir = "C:/meg/NVAR_ICC/MRI/freesurfer/", 
    initial_time = 0, 
    hemi = "both", 
    time_viewer = True
)

Using pyvistaqt 3d backend.
Using control points [0.00351941 0.00363977 0.00418828]
False


### ICC

In [ ]:
# WITHIN RUN

# Loop through band, subject, scan, and run
for band in bands: 

    # Create a dataframe just for this band
    # The dataframe will have the ICC (and CI) at each vertex
    df = pd.DataFrame(columns=["VERTEX", "ICC2", "CI"])

    for vertex in range(len(stc.vertices)):

        # Create a dataframe just for this vertex
        df_vertex = pd.DataFrame(columns=["TARGET", "RATER", "RATING"]) # subject_scan_run, valA/B, power
        # why is target subject_scan_run? because otherwise there'd be too much variability. We are trying to estimate ICC WITHIN a run

        # Loop through all runs for all sessions for all subjects
        for subject in dictionary: 
            for session in dictionary[subject]:
                for run in runs: 

                    # Read data
                    # TODO

                    # Loop through run windows; the run has 55 windows, numbered 0-54
                    for window in range(53): # Need to stop at 52 so that the final valB is = 54
                        print("Value A: " + str(window))
                        print("Value B: " + str(window+2))

                        valA = stc.data[vertex, window]
                        print(valA)
                        valB = stc.data[vertex, window+2]
                        print(valB)

                        # Add the pair of values
                        df_vertex.loc[len(df_vertex)] = ["_".join([subject, session, run]), "valA", valA]
                        df_vertex.loc[len(df_vertex)] = ["_".join([subject, session, run]), "valB", valB]

        # Compute ICC on this vertex
        results = pg.intraclass_corr(data=df_vertex, targets='TARGET', raters='RATER', ratings='RATING')
        icc2 = results.loc[1, 'ICC']
        ci = results.loc[1, "CI95%"]

    # Add to big df
    df.loc[len(df)] = [vertex, icc2, ci]

    # Save the entire thing
    with open("C:/meg/NVAR_ICC/ICC_inrun_" + band + ".pkl", "wb") as f: 
        pickle.dump(df, f)

Value A: 0
Value B: 2
-0.006775385
-0.0067887795
Value A: 1
Value B: 3
-0.00674534
-0.0052289683
Value A: 2
Value B: 4
-0.0067887795
-0.005200769
Value A: 3
Value B: 5
-0.0052289683
-0.004332451
Value A: 4
Value B: 6
-0.005200769
-0.0027767592
Value A: 5
Value B: 7
-0.004332451
-0.0027784829
Value A: 6
Value B: 8
-0.0027767592
-0.0019104307
Value A: 7
Value B: 9
-0.0027784829
-0.0031747818
Value A: 8
Value B: 10
-0.0019104307
-0.006392469
Value A: 9
Value B: 11
-0.0031747818
-0.0075479527
Value A: 10
Value B: 12
-0.006392469
-0.0066341516
Value A: 11
Value B: 13
-0.0075479527
0.00020857122
Value A: 12
Value B: 14
-0.0066341516
-0.011178154
Value A: 13
Value B: 15
0.00020857122
0.00064289366
Value A: 14
Value B: 16
-0.011178154
0.0024446482
Value A: 15
Value B: 17
0.00064289366
0.002220429
Value A: 16
Value B: 18
0.0024446482
-0.0070072617
Value A: 17
Value B: 19
0.002220429
-0.00013720278
Value A: 18
Value B: 20
-0.0070072617
0.0006007764
Value A: 19
Value B: 21
-0.00013720278
0.000492

AssertionError: Data must have at least 5 non-missing values.

In [ ]:
# ACROSS RUNS: REST1 VS REST2
# Call values from rest1 "valA" and values from rest2 "valB"

# Loop through band, subject, scan, and run
for band in bands: 

    # Create a dataframe just for this band
    # The dataframe will have the ICC (and CI) at each vertex
    df = pd.DataFrame(columns=["VERTEX", "ICC2", "CI"])

    for vertex in range(len(stc.vertices)):

        # Create a dataframe just for this vertex
        df_vertex = pd.DataFrame(columns=["TARGET", "RATER", "RATING"]) # subject_scan, valA/B, power
        # why is target subject_scan_run? because otherwise there'd be too much variability. We are trying to estimate ICC within a run 

        # Loop through all runs for all sessions for all subjects
        for subject in dictionary: 
            for session in dictionary[subject]:

                # Loop through run windows; the run has 55 windows, numbered 0-54
                for window in range(55): 

                    # Get file for runA
                    # Read data
                    # TODO
                    stc_rest1 = 
                    valA = stc_rest1.data[vertex, window]

                    # Get file for runB
                    stc_rest2 = 
                    valB = stc_rest2.data[vertex, window]

                    # Add the pair of values
                    df_vertex.loc[len(df_vertex)] = ["_".join([subject, session]), "valA", valA]
                    df_vertex.loc[len(df_vertex)] = ["_".join([subject, session]), "valB", valB]

        # Compute ICC on this vertex
        results = pg.intraclass_corr(data=df_vertex, targets='TARGET', raters='RATER', ratings='RATING')
        icc2 = results.loc[1, 'ICC']
        ci = results.loc[1, "CI95%"]

    # Add to big df
    df.loc[len(df)] = [vertex, icc2, ci]

    # Save the entire thing
    with open("C:/meg/NVAR_ICC/ICC_rest1-vs-rest2_" + band + ".pkl", "wb") as f: 
        pickle.dump(df, f)

In [ ]:
# ACROSS DAYS: TODO!!!!!!!
# Call values from rest1 "valA" and values from rest2 "valB"

# Loop through band, subject, scan, and run
for band in bands: 

    # Create a dataframe just for this band
    # The dataframe will have the ICC (and CI) at each vertex
    df = pd.DataFrame(columns=["VERTEX", "ICC2", "CI"])

    for vertex in range(len(stc.vertices)):

        # Create a dataframe just for this vertex
        df_vertex = pd.DataFrame(columns=["TARGET", "RATER", "RATING"]) # subject_scan, valA/B, power
        # why is target subject_scan_run? because otherwise there'd be too much variability. We are trying to estimate ICC within a run 

        # Loop through all runs for all sessions for all subjects
        for subject in dictionary: 


            # 
            for session in dictionary[subject]:

                # Loop through run windows; the run has 55 windows, numbered 0-54
                for window in range(55): 

                    # Get file for runA
                    # Read data
                    # TODO
                    stc_rest1 = 
                    valA = stc_rest1.data[vertex, window]

                    # Get file for runB
                    stc_rest2 = 
                    valB = stc_rest2.data[vertex, window]

                    # Add the pair of values
                    df_vertex.loc[len(df_vertex)] = ["_".join([subject, session]), "valA", valA]
                    df_vertex.loc[len(df_vertex)] = ["_".join([subject, session]), "valB", valB]

        # Compute ICC on this vertex
        results = pg.intraclass_corr(data=df_vertex, targets='TARGET', raters='RATER', ratings='RATING')
        icc2 = results.loc[1, 'ICC']
        ci = results.loc[1, "CI95%"]

    # Add to big df
    df.loc[len(df)] = [vertex, icc2, ci]

    # Save the entire thing
    with open("C:/meg/NVAR_ICC/ICC_rest1-vs-rest2_" + band + ".pkl", "wb") as f: 
        pickle.dump(df, f)

In [ ]:
# DAYS AS RATERS
"""
Shape of stc_average: 
array([[0.0053271 ],
       [0.00452035],
       [0.005533  ],
       ...,
       [0.00313858],
       [0.003465  ],
       [0.00346861]], shape=(20484, 1), dtype=float32)
"""

# Loop through bands
# for band in bands: 
band = "alpha" # test

# Vertex-wise ICC
df = pd.DataFrame(columns=["VERTEX", "ICC2", "CI"])

# Loop through vertices
# There are 20484 vertices
for vertex in range(20484): 

    df_vertex = pd.DataFrame(columns=["TARGET", "RATER", "RATING"]) # subject, session, power

    # Add the value for this vertex for average-morph for each subject/session 
    for subject in dictionary: 
        for session in dictionary[subject]: 
            
            stc_dir = os.path.join(base_dir, "sub_" + subject, session, "beamformer", "stc")
            stc_average_name = os.path.join(stc_dir, "sub_" + subject + "_average_raw_tsss_beamformer_" + band + "_stc_morphed")
            stc = mne.read_source_estimate(stc_average_name)
            power = stc.data[vertex]

            df_vertex.loc[len(df_vertex)] = [subject, session, power[0]]

    # Compute ICC on this
    results = pg.intraclass_corr(data=df_vertex, targets='TARGET', raters='RATER', ratings='RATING')
    icc2 = results.loc[1, 'ICC']
    ci = results.loc[1, "CI95%"]

    # Add to big df
    df.loc[len(df)] = [vertex, icc2, ci]

    # Save the entire thing
    with open("C:/meg/NVAR_ICC/vertexwise_ICC_" + band + ".pkl", "wb") as f: 
        pkl.dump(df, f)

C:\Users\Isabel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pingouin\reliability.py:367: RuntimeWarning: invalid value encountered in scalar divide
  l2 = n * (msb - f2u * mse) / (f2u * (k * msj + (k * n - k - n) * mse) + n * msb)


In [ ]:
# Plot whole-brain ICC maps

def plot_stc_grid(stcs, labels):
    """
    Plot a list of STCs in an nx3 grid (dorsal, right lateral, left lateral).
    
    Parameters
    ----------
    stcs : list of SourceEstimate
        One STC per row.
    labels : list of str
        Row labels. 
    """
    views = ['lateral', 'dorsal', 'lateral']
    hemis = ['rh', 'both', 'lh']

    clim = dict(kind='value', lims=[0, 0.5, 1])
    colormap = 'viridis'

    n = len(stcs)
    fig, axes = plt.subplots(n, 4, figsize=(13, 4 * n),
                         gridspec_kw=dict(width_ratios=[1, 1, 1, 0.05]))

    if n == 1:
        axes = axes[np.newaxis, :]

    for row, stc in enumerate(stcs):
        for col, (view, hemi) in enumerate(zip(views, hemis)):

            brain = stc.plot(
                subject='fsaverage',
                subjects_dir="C:/meg/NVAR_ICC_day/MRI/freesurfer/",
                hemi=hemi,
                views=view,
                clim=clim,
                colormap= colormap, 
                background='white', 
                #surface = "pial", 
                colorbar = False
            )

            img = brain.screenshot()
            axes[row, col].imshow(img)
            axes[row, col].axis('off')
            plt.close('all')


        mne.viz.plot_brain_colorbar(axes[row, 3], clim, colormap, label="ICC")

        axes[row, 0].text(-0.1, 0.5, labels[row], transform=axes[row, 0].transAxes,
                  fontsize=20, va='center', ha='right', rotation=0)

    col_titles = ['Right Lateral', 'Dorsal', 'Left Lateral']
    for col, title in enumerate(col_titles):
        axes[0, col].set_title(title, fontsize=20)

    fig.tight_layout()
    return fig

def make_and_plot_stc(files, labels): 
    """
    Given a list of paths to pickle files, makes a list of stcs, then passes them to a function for plotting
    
    Parameters
    ----------
    pkls : list of pkl, each a dataframe with columns: vertex, ICC2, CI
    labels : list of str

    """

    stcs = []

    for file in files: 

        with open(pkl, "rb") as f: 
            df = pickle.load(f)

        stc = mne.SourceEstimate(
            data = df["ICC2"].to_numpy(),
            vertices = [np.arange(0, 10242), np.arange(0, 10242)], 
            tmin=0,
            tstep=1,
            subject="fsaverage"
        )

        stcs.append(stc)
    
    return plot_stc_grid(stcs, labels)


fig = make_and_plot_stc(
    pkls=["C:/meg/NVAR_ICC_day/ICC_alpha.pkl", "C:/meg/NVAR_ICC_day/ICC_beta.pkl", "C:/meg/NVAR_ICC_day/ICC_g_low.pkl"],
    labels=['Alpha', 'Beta', 'Gamma']
)

fig.savefig('C:/meg/NVAR_ICC_day/brain_grid.png', dpi=300, bbox_inches='tight')

False
False
False
False
False
False
False
False
False
